# ***Web Scrapping pour collecter les textes :***

In [ ]:
import requests
from bs4 import BeautifulSoup
links=[]
URL = 'https://www.moroccoworldnews.com/news-2/'
i=0
while(i<9):
    p = requests.get(URL)
    soup = BeautifulSoup(p.content, 'html.parser')
    r = soup.find(class_='td-pb-span8 td-main-content')
    URLS = r.find_all('div', class_='td-block-row')
    for link in URLS:
        href=link.find('a')['href']
        links.append(href)
    URL = "https://www.moroccoworldnews.com/news-2"+"/page/"+str(i+2)+"/"
    i=i+1

In [ ]:
#ARTICLES:

ART=[]

for link in links:
  p = requests.get(link)
  soup = BeautifulSoup(p.content, 'html.parser')
  r = soup.find(class_= 'post-info-description')
  spans = r.findAll('span')
  d=""
  for i in spans:
    d=d+i.text
  ART.append(d)

In [ ]:
ART[0]

'Rabat – Morocco’s Ministry of Health has confirmed 431 new COVID-19 cases in the last 24 hours, bringing the country’s total recorded case count to 11,338 as of today at 6 p.m.The ministry reported one additional death, with the death toll totaling 217. This represents a mortality rate of 1.9%, a marginal decrease from yesterday’s 2% figure.Morocco also recorded 32 new recoveries in 24 hours, bringing the country’s total to 8,500. This brings the recovery rate to 75%. This is down slightly from yesterday’s 77.6%.The Ministry of Health reported that it is now conducting more than 20,000 daily tests.The number of active COVID-19 cases throughout Morocco continues to rise significantly on a daily basis in line with recent trends, now totaling 2,621.Today’s announcement comes as Morocco continues to gradually ease confinement measures, in line with local epidemiological developments. The government has pledged to announce any amendments to the eased measures on a weekly basis.'

# ***Pré-traitement des textes***

## 1.Harmonisation de la case:

In [ ]:
for i in range(len(ART)):
  ART[i]=ART[i].lower()

In [ ]:
ART[0]

'rabat – morocco’s ministry of health has confirmed 431 new covid-19 cases in the last 24 hours, bringing the country’s total recorded case count to 11,338 as of today at 6 p.m.the ministry reported one additional death, with the death toll totaling 217. this represents a mortality rate of 1.9%, a marginal decrease from yesterday’s 2% figure.morocco also recorded 32 new recoveries in 24 hours, bringing the country’s total to 8,500. this brings the recovery rate to 75%. this is down slightly from yesterday’s 77.6%.the ministry of health reported that it is now conducting more than 20,000 daily tests.the number of active covid-19 cases throughout morocco continues to rise significantly on a daily basis in line with recent trends, now totaling 2,621.today’s announcement comes as morocco continues to gradually ease confinement measures, in line with local epidemiological developments. the government has pledged to announce any amendments to the eased measures on a weekly basis.'

## 2.Supression de la ponctuation:

In [ ]:
import string

In [ ]:
for i in range(len(ART)):
  ART[i]=ART[i].translate(str.maketrans('', '', string.punctuation))


In [ ]:
ART[0]

'rabat – morocco’s ministry of health has confirmed 431 new covid19 cases in the last 24 hours bringing the country’s total recorded case count to 11338 as of today at 6 pmthe ministry reported one additional death with the death toll totaling 217 this represents a mortality rate of 19 a marginal decrease from yesterday’s 2 figuremorocco also recorded 32 new recoveries in 24 hours bringing the country’s total to 8500 this brings the recovery rate to 75 this is down slightly from yesterday’s 776the ministry of health reported that it is now conducting more than 20000 daily teststhe number of active covid19 cases throughout morocco continues to rise significantly on a daily basis in line with recent trends now totaling 2621today’s announcement comes as morocco continues to gradually ease confinement measures in line with local epidemiological developments the government has pledged to announce any amendments to the eased measures on a weekly basis'

## 3.Supression des caractères spéciaux et des chiffres

In [ ]:
import re

In [ ]:
for i in range(len(ART)):
   ART[i]=re.sub('[^A-Za-z]+', ' ', ART[i])

In [ ]:
ART[0]

'rabat morocco s ministry of health has confirmed new covid cases in the last hours bringing the country s total recorded case count to as of today at pmthe ministry reported one additional death with the death toll totaling this represents a mortality rate of a marginal decrease from yesterday s figuremorocco also recorded new recoveries in hours bringing the country s total to this brings the recovery rate to this is down slightly from yesterday s the ministry of health reported that it is now conducting more than daily teststhe number of active covid cases throughout morocco continues to rise significantly on a daily basis in line with recent trends now totaling today s announcement comes as morocco continues to gradually ease confinement measures in line with local epidemiological developments the government has pledged to announce any amendments to the eased measures on a weekly basis'

### Tokenization:

In [ ]:
words=[]*len(ART)
for i in range(len(ART)):
  words.append(ART[i].split())

### Supression des stops words

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop_words =set(stopwords.words('english')) 

In [ ]:
unique_words=[]*len(words)
for i in range(len(words)):
  unique_words.append([word for word in words[i] if not word in stop_words])

### Lemmatisation

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer 

In [ ]:
lemmatized_words=[]*len(unique_words)
lemmatizeur = WordNetLemmatizer()
for i in range(len(unique_words)):
  lemmatized_words.append([lemmatizeur.lemmatize(w) for w in unique_words[i]])


##Stemming

In [ ]:
from nltk.stem import PorterStemmer 


In [ ]:
stemmed_lemmatized_words=[]*len(lemmatized_words)
stemmeur=PorterStemmer() 
for i in range(len(unique_words)):
  stemmed_lemmatized_words.append([stemmeur.stem(w) for w in lemmatized_words[i]])


# ***Execution des models LSA et LDA :*** 


### Bag of Words et la matrice article-termes

The Bag of Words :

In [ ]:
from itertools import chain
B_of_words = list(set(chain.from_iterable(stemmed_lemmatized_words)))

The article-terme matrix :

In [ ]:
nbr_of_terms = []*len(ART)

for i in range(len(ART)):

  nbr_of_terms.append(dict.fromkeys(B_of_words, 0))
  for term in stemmed_lemmatized_words[i]:
      nbr_of_terms[i][term] += 1

In [ ]:
import pandas as pd

Matrix_article_term = pd.DataFrame(nbr_of_terms)
Matrix_article_term

,regist,generos,data,passport,tripartit,incur,strength,abolish,dignitari,fakih,iffou,inadvert,khashoggi,redal,gran,sandstorm,loss,talk,sister,selfisolateamzazi,increasingli,netizen,reshap,done,polisario,timo,belief,lawyer,bouritaspeak,journal,texa,express,could,sword,luc,impress,kilogram,eddahab,attent,corpor,...,increment,wherea,khalifath,review,part,contrast,customari,colombian,robert,tobroukbas,quarantineswhil,patientther,came,enabl,usback,materi,debat,penal,resign,billion,sundayagadirtangi,take,dortmund,analyz,glasgow,shop,deaf,hirak,entrepreneurship,airplan,ibn,marketsth,oujdaal,abroadspeak,forefront,cuisin,provinc,fli,flee,capacityread
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
114,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
115,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**La matrice article-term avec la librairie gensim :**

In [ ]:
from gensim import corpora, models

AllWords = corpora.Dictionary(stemmed_lemmatized_words)
B_of_words2 = [AllWords.doc2bow(SLW) for SLW in stemmed_lemmatized_words]


## 1 - Le modèle LSA :

In [ ]:
import gensim
from gensim import corpora, models


Les Données 


In [ ]:
Terms_LSA = corpora.Dictionary(stemmed_lemmatized_words)
Bag_of_Words = [AllWords.doc2bow(SLW) for SLW in stemmed_lemmatized_words]

**The article-terme matrix :**

In [ ]:
from gensim.models import LsiModel
LSAmodel = LsiModel(Bag_of_Words , num_topics=2, id2word =Terms_LSA )  





**RESULAT**

In [ ]:
print(LSAmodel.print_topics(num_topics=2, num_words=10))

[(0, '-0.491*"morocco" + -0.287*"moroccan" + -0.275*"countri" + -0.170*"intern" + -0.165*"algeria" + -0.151*"said" + -0.135*"govern" + -0.130*"covid" + -0.120*"report" + -0.115*"also"'), (1, '0.319*"donkey" + -0.293*"intern" + -0.268*"amnesti" + -0.225*"report" + -0.214*"use" + -0.177*"spywar" + 0.172*"anim" + 0.152*"care" + 0.145*"work" + 0.128*"moulay"')]


## 2 - Le modèle LDA (avec le bag of Words) sur la matrice :

In [ ]:
import gensim
LDAmodel = gensim.models.ldamodel.LdaModel(B_of_words2, num_topics=2, id2word = AllWords, passes=30)

**Les resultats num 1 :**

In [ ]:
print(LDAmodel.print_topics(num_topics=2, num_words=10))

[(0, '0.018*"morocco" + 0.011*"countri" + 0.008*"covid" + 0.007*"moroccan" + 0.006*"said" + 0.005*"case" + 0.005*"health" + 0.005*"also" + 0.005*"african" + 0.005*"intern"'), (1, '0.010*"morocco" + 0.010*"moroccan" + 0.006*"countri" + 0.006*"june" + 0.006*"also" + 0.005*"repatri" + 0.005*"includ" + 0.005*"oper" + 0.004*"covid" + 0.004*"said"')]


## 3 - le modèle LDA avec TF-IDF :

In [ ]:
TFidf = models.TfidfModel(B_of_words2)
AllWords_TFidf = TFidf[B_of_words2]

In [ ]:
LDAmodel_TFidf = gensim.models.LdaMulticore(AllWords_TFidf, num_topics=2, id2word=AllWords, passes=2, workers=3)


**Les resultats num 2 :**

In [ ]:
print(LDAmodel_TFidf.print_topics(num_topics=2, num_words=10))

[(0, '0.001*"repatri" + 0.001*"african" + 0.001*"strand" + 0.001*"king" + 0.001*"child" + 0.001*"medic" + 0.001*"zone" + 0.001*"algeria" + 0.001*"aid" + 0.001*"initi"'), (1, '0.001*"case" + 0.001*"region" + 0.001*"measur" + 0.001*"ministri" + 0.001*"health" + 0.001*"total" + 0.001*"hour" + 0.001*"record" + 0.001*"new" + 0.001*"regist"')]


# **Évaluation des performances :**

In [ ]:
ART[99]

'rabat beijing did not register any new covid cases for nearly two months until that streak broke on june since then a total of locals have tested positive for the virus the chinese government had started easing the nationwide lockdown in april and public spaces such as schools shops and offices resumed operations this occurred during a time when much of the world was observing china as a postpandemic examplemany anticipated a second coronavirus wave in china and after months of eased restrictions coronavirus reappeared in beijing one of the most populous cities in the worldbeijing officials immediately sealed the area where they detected cases and began mass covid testingauthorities confirmed that the new cases originated in a wholesale market xinfadi the largest of its kind in asia the market provides supplies for tens of thousands of people every day seeing cases detected in a market of this scale has instilled fear not only in the hearts of citizens but also the officials of beijin

In [ ]:
text_evaluation = B_of_words2[99] 

**1-Évaluation des performances en classant un exemple de document à l'aide du modèle LSA**

In [ ]:
for index, score in sorted(LSAmodel[text_evaluation], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, LSAmodel.print_topic(index, 9)))


Score: 1.1030387184404589	 
Topic: 0.319*"donkey" + -0.293*"intern" + -0.268*"amnesti" + -0.225*"report" + -0.214*"use" + -0.177*"spywar" + 0.172*"anim" + 0.152*"care" + 0.145*"work"

Score: -4.046206287612737	 
Topic: -0.491*"morocco" + -0.287*"moroccan" + -0.275*"countri" + -0.170*"intern" + -0.165*"algeria" + -0.151*"said" + -0.135*"govern" + -0.130*"covid" + -0.120*"report"



**2-Évaluation des performances en classant un exemple de document à l'aide du modèle LDA Bag of Words**

In [ ]:
for index, score in sorted(LDAmodel[text_evaluation], key=lambda tup: -1*tup[1]):
    print("\nScore : {} Sujet : {}".format(score, LDAmodel.print_topic(index, 9)))


Score : 0.8939690589904785 Sujet : 0.018*"morocco" + 0.011*"countri" + 0.008*"covid" + 0.007*"moroccan" + 0.006*"said" + 0.005*"case" + 0.005*"health" + 0.005*"also" + 0.005*"african"

Score : 0.10603100061416626 Sujet : 0.010*"morocco" + 0.010*"moroccan" + 0.006*"countri" + 0.006*"june" + 0.006*"also" + 0.005*"repatri" + 0.005*"includ" + 0.005*"oper" + 0.004*"covid"


**3-Évaluation des performances en classant un exemple de document à l'aide du modèle LDA TF-IDF**

In [ ]:
for index, score in sorted(LDAmodel_TFidf[text_evaluation], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, LDAmodel_TFidf.print_topic(index, 9)))


Score: 0.9951769113540649	 
Topic: 0.001*"repatri" + 0.001*"african" + 0.001*"strand" + 0.001*"king" + 0.001*"child" + 0.001*"medic" + 0.001*"zone" + 0.001*"algeria" + 0.001*"aid"


# **Tester les modèle sur un nouveau article :**

Text de test :

In [ ]:
Text_Test="Rabat – Morocco’s Ministry of Health today confirmed 25 new COVID-19 cases, bringing the total number of cases to 8,533 as of 10 a.m. on June 11.  The ministry reported five new recoveries. Morocco’s total number of coronavirus recoveries is now 7,570. The recovery rate stands at 88.7%.   The health ministry announced no fatalities this morning.     The death toll remains at 211, with a 2.5% fatality rate.   The Casablanca-Settat and Marrakech-Safi regions have recorded the highest COVID-19 case counts, with 33.69 % of Morocco’s total cases and 18%, respectively.  The Tangier-Tetouan-Al Hoceima ha 14.36% cases, followed by Fez-Meknes ( 12.11%), Rabat-Sale-Kenitra ( 9.21%), Draa-Tafilalet ( 6.87%), and Oriental regions ( 2.30 %).  Beni Mellal-Khenifra has 1.49% of the cases, followed by Souss-Massa (1.05%), Guelmim-Oued Noun (0.54 %), Dakhla-Oued Ed Dahab (0.06 %), and Laayoune-Sakia El Hamra (0.06 %).     Since February, 355,817 suspected cases have tested negative from COVID-19.  On June 8, Morocco announced its decision to extend the state of emergency to July 10. The country also eased lockdown in several regions depending on the epidemiological situation in each region.  Citizens and residents living in Zone 1 can now leave their homes without special authorization, while citizens living in Zone 2 remain under strict lockdown."

Pre-traitement du nouveau text :

In [ ]:
Text_Test = Text_Test.lower()
Text_Test = Text_Test.translate(str.maketrans('', '', string.punctuation))
Text_Test=re.sub('[^A-Za-z]+', ' ', Text_Test)
Text_Test = Text_Test.split()
TestText = [word for word in Text_Test if not word in stop_words]
TestText = [lemmatizeur.lemmatize(w) for w in TestText]
TestText = [stemmeur.stem(w) for w in TestText]
boW_Text_Test = AllWords.doc2bow(TestText)

## Test des modèles :

1 - LSA :

In [ ]:
for ind, score in sorted(LSAmodel[boW_Text_Test], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, LSAmodel.print_topic(ind, 11)))

Score: 0.8482431560471563	 Topic: 0.319*"donkey" + -0.293*"intern" + -0.268*"amnesti" + -0.225*"report" + -0.214*"use" + -0.177*"spywar" + 0.172*"anim" + 0.152*"care" + 0.145*"work" + 0.128*"moulay" + 0.124*"button"
Score: -5.5660090035848295	 Topic: -0.491*"morocco" + -0.287*"moroccan" + -0.275*"countri" + -0.170*"intern" + -0.165*"algeria" + -0.151*"said" + -0.135*"govern" + -0.130*"covid" + -0.120*"report" + -0.115*"also" + -0.102*"use"


2 - LDA avec BoW :

In [ ]:
for ind, score in sorted(LDAmodel[boW_Text_Test], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, LDAmodel.print_topic(ind, 11)))

Score: 0.8412602543830872	 Topic: 0.018*"morocco" + 0.011*"countri" + 0.008*"covid" + 0.007*"moroccan" + 0.006*"said" + 0.005*"case" + 0.005*"health" + 0.005*"also" + 0.005*"african" + 0.005*"intern" + 0.004*"algeria"
Score: 0.15873977541923523	 Topic: 0.010*"morocco" + 0.010*"moroccan" + 0.006*"countri" + 0.006*"june" + 0.006*"also" + 0.005*"repatri" + 0.005*"includ" + 0.005*"oper" + 0.004*"covid" + 0.004*"said" + 0.004*"nation"


3 - LDA avec TF-IDF :


In [ ]:
for ind, score in sorted(LDAmodel_TFidf[boW_Text_Test], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, LDAmodel_TFidf.print_topic(ind, 11)))

Score: 0.9887500405311584	 Topic: 0.001*"case" + 0.001*"region" + 0.001*"measur" + 0.001*"ministri" + 0.001*"health" + 0.001*"total" + 0.001*"hour" + 0.001*"record" + 0.001*"new" + 0.001*"regist" + 0.000*"report"
Score: 0.011249926872551441	 Topic: 0.001*"repatri" + 0.001*"african" + 0.001*"strand" + 0.001*"king" + 0.001*"child" + 0.001*"medic" + 0.001*"zone" + 0.001*"algeria" + 0.001*"aid" + 0.001*"initi" + 0.001*"moham"
